Tell me dear reader, who among us, while gazing in awe at the vast emptiness of the star-filled night sky, hasn't wondered:
how do gradient boosting trees implement multi-class classification?
Today, we'll unravel this mystery by reviewing the original multi-class classification gradient boosting algorithm from the classic Friedman paper and implementing one for ourselves in python.

If you need a refresher on gradient boosting, then start with 
[gradient boosting regression model from scratch](/posts/gradient-boosting-machine-from-scratch/).,
which is the first post in this epic [series on gradient boosting](/gradient-boosting-series/).




## Coming to terms with multi-class classification

Let's establish our terminology and notation  for multi-class classification.
First off, *binary* classification means that our target has two discrete levels, e.g. true or false.
In *multi-class* classification problems our target has 3 or more possible levels, e.g. setosa, virginica, and versicolor.
There are two ways that we'll be representing these discrete data: with integer encoding and with one hot encoding.
Integer encoding maps each text label to a specific integer.
One hot encoding maps each text label to a vector with a 1 in a specific position and 0's everywhere else.
Here's how the three labels setosa, virginica, and versicolor from the [iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set)
might get encoded.


| text label | integer encoding | one hot encoding
| --- | --- | --- |
| setosa | 0 | (1, 0, 0) |
| virginica | 1 | (0, 1, 0) |
| versicolor | 2 | (0, 0, 1) |

While describing things in the mathematical notation, we'll be working with the one hot encoded representation of the labels.
In general when the target has $K$ discrete levels,
the one hot encoded representation of a single training example is the vector $\{y_1,y_2,\dots,y_K\}$, where $y_k \in \{0,1\}$.
For compactness, we'll write this set like $\{y_k\}_1^K$.

Our model is going to take a feature vector $\mathbf{x}$ of a specified length as input, and it will return a length-$K$ vector of probabilities $\{p_k(\mathbf{x})\}_1^K$, where $p_k(\mathbf{x})$ gives the probability that $y_k=1$, and where the probabilities sum to 1 $\sum_{k=1}^K p_k(\mathbf{x})=1$.

To ensure that model outputs are valid probabilities, i.e. $0 \le p_k(\mathbf{x}) \le 1$ and $\sum_{k=1}^K p_k(\mathbf{x})=1$, we'll use the notion of a link function which you might have seen in generalized linear modeling.
Recall that a link function takes us from probability space to link space,
and an inverse link function takes us from link space to probability space.
Here we use the symmetric multiple logistic function as the link and the softmax as the inverse link, i.e.

$$ p_k(\mathbf{x}) = \text{softmax}_k(\{F_k(\mathbf{x})\}_1^K) 
    = \frac{e^{F_k(\mathbf{x})}}{\sum_{l=1}^K e^{F_l(\mathbf{x})}}$$

where $\{F_k(\mathbf{x})\}_1^K$ is the length-$K$ vector of raw model outputs.

To make all this a bit more concrete, let's consider how this might look for a few rows of the iris data where we have $K=3$ discrete labels and for a model that uses a length-2 feature vector $\mathbf{x}$ with (petal length, petal width).

| text label | $y_1$ | $y_2$ | $y_3$ | $\mathbf{x}$ | $F_1(\mathbf{x})$ | $F_2(\mathbf{x})$ | $F_3(\mathbf{x})$ | $p_1(\mathbf{x})$ | $p_2(\mathbf{x})$ | $p_3(\mathbf{x})$ |
|---|---|---|---|---|---|---|---|---|---|---|
| setosa | 1 | 0 | 0 | (1.4, 0.2) | 1.9 | 1.1 | -0.5 | 0.65 | 0.29 | 0.06 |
| versicolor | 0 | 0 | 1 | (4.7, 1.4) | -0.1 | 0.7 | 2.0 | 0.09 | 0.20 | 0.71 |
| virginica | 0 | 1 | 0 | (5.8, 2.2) | 0.2 | 0.7 | -1.0 | 0.34 | 0.56 | 0.10 |

In [10]:
import numpy as np
def _softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=1).reshape(-1, 1)

_softmax(np.array([1.9, 1.1, -0.5]).reshape(1,-1))
_softmax(np.array([-0.1, 0.7, 2.0]).reshape(1,-1))
_softmax(np.array([0.2, 0.7, -1.0]).reshape(1,-1))

array([[0.33899276, 0.55890458, 0.10210266]])

## The Algorithm

By now you know the general pattern for gradient boosting algorithms; if not, go review the [generic gradient boost algorithm](/posts/gradient-boosting-machine-with-any-loss-function/).

1. Set the initial model predictions.

1. Repeat the following for each boosting round.

1. &nbsp;&nbsp;&nbsp;&nbsp;
Compute the pseudo residuals (negative gradients).

1. &nbsp;&nbsp;&nbsp;&nbsp;
Train a regression tree to predict the pseudo residuals.

1. &nbsp;&nbsp;&nbsp;&nbsp;
Adjust the tree's predicted values to optimize the overall objective.

1. &nbsp;&nbsp;&nbsp;&nbsp;
Update the composite model by adding this new tree to it.

The only wrinkle in multi-class classification is that at each boosting round, instead of training a single tree to predict pseudo residuals, we'll need to train $K$ trees, one for each level of the target variable.
Let's take a look at the details for each of these steps.

### Initial predictions

The simplest way to do this is to set the initial probabilities to $1/K$, i.e. $p_k(\mathbf{x})=1/K$ for $k=1,2,\dots,K$, which implies $F_k(\mathbf{x})=0$.

### Pseudo Residuals

For each example in the training dataset, the pseudo residual is the negative gradient of the objective function with respect to the model prediction.
The objective function for multi-class classification is the Multinomial Negative Log Likelihood.
For a single observation, the objective is

$$ J(\{ y_k, p_k(\mathbf{x}) \}_1^K) = -\sum_{k=1}^K y_k \log p_k(\mathbf{x}) $$

We can rewrite the objective in terms of our link space raw model output $F$ like this.

$$ J(\{ y_k, F_k(\mathbf{x}) \}_1^K) = -\sum_{k=1}^K y_k \log \frac{e^{F_k(\mathbf{x})}}{\sum_{l=1}^K e^{F_l(\mathbf{x})}}$$

The negative gradient of the objective with respect to raw model prediction $F_k(\mathbf{x}_i)$ for training example $i$ is given by

$$ r_{ik} = -J'(F_k(\mathbf{x}_i)) = -\left[ \frac{\partial J(\{ y_{il}, F_l(\mathbf{x_i})\}_{l=1}^K)}{\partial F_k(\mathbf{x}_i) } \right] 
=y_{ik} - p_{k}(\mathbf{x}_i)$$

You can take a look at the [derivation](https://towardsdatascience.com/derivative-of-the-softmax-function-and-the-categorical-cross-entropy-loss-ffceefc081d1)
if you're curious how to work it out yourself.
Note that this formula has a nice intuition.
When $y_{ik}=1$, if predicted probability $p_k(\mathbf{x}_i)$ is terrible and close to 0, then pseudo residual will be positive, and the next boosting round will try to increase the predicted probability.
Otherwise if the predicted probability is already good and close to 1, the pseudo residual will be close to 0 and the next boosting round won't change the predicted probability.

### Boosting tree predicted values

After training a regression tree to predict the pseudo residuals,  we need to adjust the predicted values in its terminal nodes to optimize the overall objective function.

In the paper, he actually specifies finding the optimal value using a numerical optimization routine like line search.

$$ v = \text{argmin}_v \sum_{i \in t} J(y_{i}, F(\mathbf{x}_i) + v) $$

where $t$ is the set of samples falling into this terminal node.

In the scikit-learn implementation of gradient boosting classification, the authors instead use a single Newton descent step to approximate the optimal predicted value for each terminal node.
See code and comments for the function `_update_terminal_regions` in the scikit-learn gradient boosting module.
The updated value is computed as 

$$ v = -\frac{\sum_{i \in t} J'(F_k(\mathbf{x}_i))}{\sum_{i \in t} J''(F_k(\mathbf{x}_i))} $$

We already found the first derivative or gradient of the objective, so we just need to calculate the second derivative or hessian.

$$ J''(F_k(\mathbf{x}_i)) = 
\left[ \frac{\partial J(\{ y_{il}, F_l(\mathbf{x_i})\}_{l=1}^K)}{\partial ^2 F_k(\mathbf{x}_i) } \right]
= p_k(\mathbf{x}_i) (1 - p_k(\mathbf{x}_i))
$$

We now have all the ingredients we need to implement gradient boosting multi-class classification from scratch.

In [ ]:
import numpy as np
import pandas as pd 
from sklearn.tree import DecisionTreeRegressor 
from sklearn.preprocessing import OneHotEncoder

class GBClassifier():
    '''Gradient Boosting Classifier.
    
    Parameters
    ----------
    n_estimators : int
        number of boosting rounds
        
    learning_rate : float
        learning rate hyperparameter
        
    max_depth : int
        maximum tree depth
    '''
    
    def __init__(self, n_estimators, learning_rate=0.1, max_depth=1):
        self.n_estimators=n_estimators; 
        self.learning_rate=learning_rate
        self.max_depth=max_depth;
    
    def fit(self, X, y):
        '''Fit the GBM using the specified loss function.
        
        Parameters
        ----------
        X : ndarray of size (number observations, number features)
            design matrix
            
        y : ndarray of size (number observations,)
            target labels in {0,1,...,k-1}
        '''
        
        self.n_classes = pd.Series(y).nunique()
        y_ohe = self._one_hot_encode_labels(y)

        raw_predictions = np.zeros(shape=y_ohe.shape)
        probabilities = self._softmax(raw_predictions)
        self.boosters = []
        for m in range(self.n_estimators):
            class_trees = []
            for k in range(self.n_classes):
                negative_gradients = self._negative_gradients(y_ohe[:, k], probabilities[:, k])
                hessians = self._hessians(probabilities[:, k])
                tree = DecisionTreeRegressor(max_depth=self.max_depth)
                tree.fit(X, negative_gradients);
                self._update_terminal_nodes(tree, X, negative_gradients, hessians)
                raw_predictions[:, k] += self.learning_rate * tree.predict(X)
                probabilities = self._softmax(raw_predictions)
                class_trees.append(tree)
            self.boosters.append(class_trees)
    
    def _one_hot_encode_labels(self, y):
        if isinstance(y, pd.Series): y = y.values
        ohe = OneHotEncoder()
        y_ohe = ohe.fit_transform(y.reshape(-1, 1)).toarray()
        return y_ohe
        
    def _negative_gradients(self, y, p):
        return y - p
    
    def _hessians(self, p): 
        return p * (1 - p)

    def _softmax(self, x):
        return np.exp(x) / np.sum(np.exp(x), axis=1).reshape(-1, 1)
        
    def _update_terminal_nodes(self, tree, X, negative_gradients, hessians):
        '''Update the terminal node predicted values'''
        # terminal node id's
        leaf_nodes = np.nonzero(tree.tree_.children_left == -1)[0]
        # compute leaf for each sample in ``X``.
        leaf_node_for_each_sample = tree.apply(X)
        for leaf in leaf_nodes:
            samples_in_this_leaf = np.where(leaf_node_for_each_sample == leaf)[0]
            negative_gradients_in_leaf = negative_gradients.take(samples_in_this_leaf, axis=0)
            hessians_in_leaf = hessians.take(samples_in_this_leaf, axis=0)
            val = np.sum(negative_gradients_in_leaf) / np.sum(hessians_in_leaf)
            tree.tree_.value[leaf, 0, 0] = val
          
    def predict_proba(self, X):
        '''Generate probability predictions for the given input data.'''
        raw_predictions =  np.zeros(shape=(X.shape[0], self.n_classes))
        for k in range(self.n_classes):
            for booster in self.boosters:
                raw_predictions[:, k] += booster[k].predict(X)
        probabilities = self._softmax(raw_predictions)
        return probabilities
        
    def predict(self, X):
        '''Generate predicted labels (as 1-d array)'''
        probabilities = self.predict_proba(X)
        return np.argmax(probabilities, axis=1)

## Datasets

In [ ]:
# from sklearn.datasets import load_iris

# dbunch = load_iris(as_frame=True)
# df = dbunch['frame']
# target = 'target'
# features = dbunch['feature_names']
# X = df[features]
# y = df[target]

In [237]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = make_classification(n_samples=10000, 
                           n_classes=5, 
                           n_features=20,
                           n_informative=10,
                           random_state=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [241]:
gb_sklearn = GradientBoostingClassifier(n_estimators=10, max_depth=1)
gb_sklearn.fit(X_train, y_train)
accuracy_score(y_test, gb_sklearn.predict(X_test))

0.444

In [242]:
gb_scratch = GBClassifier(n_estimators=10, max_depth=1)
gb_scratch.fit(X_train, y_train)
accuracy_score(y_test, gb_scratch.predict(X_test))

0.4524